In [0]:
%run ../../../bootstrap

In [0]:
import sys
import os
import importlib.util
from src.common.init_tables import schema_to_sql, create_table

tables_path = os.path.abspath(os.path.join(os.getcwd(), "../tables"))
tables_infos = []

for fname in os.listdir(tables_path):
    if fname.endswith(".py") and not fname.startswith("__"):
        fpath = os.path.join(tables_path, fname)
        module_name = fname[:-3]
        spec = importlib.util.spec_from_file_location(module_name, fpath)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        if hasattr(module, "get_table_info"):
            info = module.get_table_info()
            tables_infos.append(info)
            print(f"Tabla {info['table']} lista para procesar.")

for info in tables_infos:
    catalog, database_name, table_name = info["table"].split(".")
    table_schema_sql = schema_to_sql(info["schema"])
    table_location = info["path"]
    table_comment = info["description"]
    partition = ", ".join(info["partition_by"]) if info.get("partition_by") else None
    primary_key = ", ".join(info["primary_key"]) if info.get("primary_key") else None

    create_table(
        database_name=database_name,
        table_name=table_name,
        table_schema=table_schema_sql,
        table_location=table_location,
        table_comment=table_comment,
        partition=partition,
        primary_key=primary_key
    )